In [1]:
# Necessary imports of functions and dictionaries
from dictionaries import *
from sample_sheet_functions import *

# User downloads sample sheet
*Requires User* 
- to pick for which tables 
- how many if 96-wells
- Project name, experiment name, additional comments

_To add_:
- User can choose csv or xlsx (xlsx is going to be coloured)


In [2]:
# Essential input
plate_wells_number_options = ['24', '48', '96']
how_many_96_plates_options = [1, 'up to', 8]
project_name = ''

# Non essential
experiment_name = ''
additional_comments = ''
file_format = ['csv', 'xlsx'] #def csv

In [3]:
# Simulating user input up to this point
user_plate = '96'
user_plate_number = 5
user_project_name = 'my_project'
user_experiment_name = 'my_experiment'
user_additional_comments = 'my_comments'
user_file_format = 'csv'

In [4]:
# Add option for the CSV format
def wrapper_make_sample_sheet(plate_type, how_many, project_name, file_format='csv'):
    sample_sheet = None
    filename = ''
    # If the plate type is 96, call the function for 96, if it's not check, which one and give the right boolean to the 24/48 function
    if plate_type == '96':
        filename=f'96_well_template_{project_name}.csv'
        sample_sheet = make_96_well_sample_sheet(filename, how_many)
    else:
        if plate_type == '24':
            filename=f'24_well_template_{project_name}.csv'
            sample_sheet = make_24_48_well_sample_sheet(filename, True)
        elif plate_type == '48':
            filename=f'48_well_template_{project_name}.csv'
            sample_sheet = make_24_48_well_sample_sheet(filename, False)
        else:
            print('I DO NOT RECOGNISE THIS PLATE TYPE')
            raise Exception('Wrong plate format required!')

    #Check that file does not exist already
    if os.path.isfile(filename):
        raise Exception('This file already exists') 

    # Save in file the string for the sample sheet (this will probably need to be modified for Dash, not sure how it handles the download of files yet)
    with open(filename, 'w') as file:
        file.write(sample_sheet)
    


In [5]:
# Execute make sample sheet with user specified variables
wrapper_make_sample_sheet(user_plate, user_plate_number, user_project_name)

# User reuploads sample sheet
#### Displays the layout of the uploaded sample sheet(s) to the user
*Persisting from before/can be updated*
- Experiment_name
- Additional_comments

*Requires user*
- Pick plate flavours

_In background_:
- Checks number of samples
- Checks number of blanks/controls
- Checks no samples start with underscore (_)

_To add_:
- User can upload CSV or XLSX

_Processes:_
- Read file in, extract different tables and display to users
- Associate each table to a plate flavour
- Associate dictionaries and produce generic melted sample sheet

In [6]:
# User reuploaded file, in this case the file name is:
user_file_name_on_reupload = '96_well_template_4_plates.csv'

# Persisting from before
user_experiment_name = user_experiment_name
user_additional_comments = user_additional_comments

In [7]:
# The file gets read, and the table(s) get loaded on pandas. The tables are then displayed to the user, where they're prompted to select the plate flavours

#Here add read extension and different read in methods
sheet = pd.read_csv(user_file_name_on_reupload, index_col=0)

tables = get_tables(sheet)

print(f'I need you to select exactly {len(tables)} plate flavours')

I need you to select exactly 4 plate flavours


In [8]:
# Here the user inputs what flavours he wants for the tables, in this case:
user_flavours_selection = ['G', 'H', 'I', 'F'] 

# In the next section the user has to select the machinery so:
possible_machinery_selection = ['hi_seq', 'mini_seq', 'mi_seq', 'next_seq']

user_machinery_selection = 'next_seq' #(from radio button on dash)

In [9]:
# Function to combine header and melted table
def combine_into_sample_sheet(tables, selected_flavours, machinery_selection, user_project_name, user_experiment_name = '', user_additional_comments=''):
    header = ''
    dfs = []
    filename = ''
    if machinery_selection == 'hi_seq':
        header = make_hiseq_header()
        dfs = loop_and_stack(tables, selected_flavours)
        filename = f'{user_project_name}_SR_HS.csv'
    
    elif machinery_selection == 'mini_seq':
        header = make_miniseq_header(user_experiment_name)
        dfs = loop_and_stack(tables, selected_flavours, mini_seq=True)
        filename = f'{user_project_name}_MiniSeq.csv'

    elif machinery_selection == 'mi_seq':
        header = make_miseq_header(user_project_name, user_experiment_name, user_additional_comments)
        dfs = loop_and_stack(tables, selected_flavours)
        filename = f'{user_project_name}_SR_MS.csv'

    elif machinery_selection == 'next_seq':
        header = make_nextseq_header(user_project_name, user_experiment_name, user_additional_comments)
        dfs = loop_and_stack(tables, selected_flavours, next_seq=True)
        filename = f'{user_project_name}_PE_NS.csv'    
    
    else:
        print('Something went wrong, match not made')
        raise Exception('This machinery is not supported yet!')


    # Need to concatenate, remove headers, re-index and remove standard index from file
    merged_dfs = concatenate_and_reindex(dfs)     
    
    # Write header and dfs to csv file
    with open(filename, 'w') as file:
        file.write(header)

        merged_dfs.to_csv(file, index=False, header=False)

    print('DONE') # Or return some kind of feedback to dash





combine_into_sample_sheet(tables, user_flavours_selection, user_machinery_selection, user_project_name, user_experiment_name, user_additional_comments)




DONE


# Prompt the user for download
*Requires user*
- Pick which machine they need

_Processes_:
- Append header to melted table